In [261]:
import psycopg2 as pg
import pickle
import glob
import pandas.io.sql as psql
import pandas as pd
import pprint
from IPython.display import clear_output
import os
from sklearn import preprocessing
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split

In [12]:
# connect to the database
try:
    conn = pg.connect("dbname='DP' user='postgres' host='localhost' password='root'")
except:
    print ("Unable to connect to the database")

### for every comment made by (suspicious) user calculate additional attributes

    - (numof_older_comments)        \# of comments made before his comment was made
    - (numof_parent_comments)       \# of comments with at least one reply
    - (numof_replies)               \# of total replies in submission
    - (numof_authors)               \# of unique authors in submisission to the point user made comment (including suspicious user that made comment we are calculating attributes for)
    - (numof_author_older_comments) \# of comments by (supsicious) users (BEFORE current comment)- this will tell us how long user stayed in discussion
#### future
- \# of comments in most developed comment thread
- \# of comments with score greater than threshold X


### other attributes that are already present
- score
- controversiality
- created_utc - from this we can calculate how often user posted, how long was greatest gap between his posts, how active he was, did he slept at all ? etc.
- subreddit
- edited

### all attributes will be grouped into interval by specific amount of days


In [55]:
#create empty dataframe
#suspicious - 1 for suspicious user; 0 - for regular user
columns = ['comment_id', 'suspicious', 'numof_older_comments','numof_parent_comments', 'numof_replies', 'numof_authors', 'numof_author_older_comments']
dataFrame = pd.DataFrame(columns=columns)

# caluclate attributes for suspicious users

In [56]:
pathForSuspiciousComments = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\suspicious\\'

i = 0 
j = 0
#for all comments made by all suspicious user - for every suspicious user
for file in glob.glob(pathForSuspiciousComments + '*_comments.pkl'):
    with open(file, 'rb') as f: 
        suspiciousAccountComments = pickle.load(f)
    clear_output()
    print(j)
    j += 1
    for comment in suspiciousAccountComments:
        # number of comments made before his (examined user) comment was made
        numof_older_comments = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
        # number of comments with at least one reply (- 1 because parent can be link_id as well)
        numof_parent_comments = psql.read_sql("SELECT COUNT(DISTINCT parent_id) - 1 AS count FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
        # number of total replies in submission
        numof_replies = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' and parent_id != '" + comment['link_id'] + "'", conn)
        # number of unique authors in submisission to the point user made comment (including suspicious user that made comment we are calculating attributes for)
        numof_authors = psql.read_sql("SELECT COUNT(DISTINCT author) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
        # number of comments by supsicious users (BEFORE current comment)- this will tell us how long user stayed in discussion
        numof_author_older_comments = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' AND author = '" + comment['author'] + "'", conn)
    
        # 1 - for suspicious user
        dataFrame.loc[i] = [comment['id'], 1, numof_older_comments['count'][0], numof_parent_comments['count'][0], numof_replies['count'][0], numof_authors['count'][0], numof_author_older_comments['count'][0]]
        i += 1

940


# calculate attributes for regular users

In [82]:
pathForRegularComments = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\normal\\'

#ignore blacklisted regular users 

with open('D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\blacklistedRegularUsers.pkl', 'rb') as f: 
    blacklistedRegularUsers = pickle.load(f)

j = 0
#for all comments made by all regular user - for every regular user
for file in glob.glob(pathForRegularComments + '*_comments.pkl'):
    with open(file, 'rb') as f: 
        regularAccountComments = pickle.load(f)
    
    #if regular user is not blacklisted
    if (os.path.basename(file)[:-13] not in blacklistedRegularUsers):  
        clear_output()
        print(j)
        j += 1
        for comment in regularAccountComments:
            # number of comments made before his (examined user) comment was made
            numof_older_comments = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
            # number of comments with at least one reply (- 1 because parent can be link_id as well)
            numof_parent_comments = psql.read_sql("SELECT COUNT(DISTINCT parent_id) - 1 AS count FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
            # number of total replies in submission
            numof_replies = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' and parent_id != '" + comment['link_id'] + "'", conn)
            # number of unique authors in submisission to the point user made comment (including regular user that made comment we are calculating attributes for)
            numof_authors = psql.read_sql("SELECT COUNT(DISTINCT author) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
            # number of comments by supsicious users (BEFORE current comment)- this will tell us how long user stayed in discussion
            numof_author_older_comments = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' AND author = '" + comment['author'] + "'", conn)

            dataFrame.loc[i] = [comment['id'], 0, numof_older_comments['count'][0], numof_parent_comments['count'][0], numof_replies['count'][0], numof_authors['count'][0], numof_author_older_comments['count'][0]]
            i += 1

237


# save dataframe 

In [85]:
#save complete submissions
path = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\'
filename = 'attributes.pkl'
with open(path + filename, 'wb') as f:
    pickle.dump(dataFrame, f)

# calculation of individual attributes

In [8]:
for comment in suspiciousAccountComments:
    # number of comments made before his (examined user) comment was made
    df = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
    df
    break

In [75]:
for comment in suspiciousAccountComments:
    # number of comments with at least one reply (- 1 because parent can be link_id as well)
    df = psql.read_sql("SELECT COUNT(DISTINCT parent_id) - 1 AS count FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
    df
    break

In [81]:
df['count'][0]

14

In [59]:
for comment in suspiciousAccountComments:
    # number of total replies in submission
    df = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' and parent_id != '" + comment['link_id'] + "'", conn)
    df
    break

In [60]:
df

,count
0,25


In [63]:
for comment in suspiciousAccountComments:
    # number of unique authors in submisission to the point user made comment (including suspicious user that made comment we are calculating attributes for)
    df = psql.read_sql("SELECT COUNT(DISTINCT author) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "'", conn)
    df
    break

In [64]:
df

,count
0,17


In [73]:
for comment in suspiciousAccountComments:
    # number of comments by supsicious users (BEFORE current comment)- this will tell us how long user stayed in discussion
    df = psql.read_sql("SELECT count(id) FROM comments WHERE created_utc < " + str(comment['created_utc']) + " AND link_id = '" + comment['link_id'] + "' AND author = '" + comment['author'] + "'", conn)
    df
    break

In [74]:
df

,count
0,2
